In [47]:
import numpy as np

with open('file.txt', 'r') as file:
    lines = file.read().strip().split('\n')

for line in lines:
    print(line)

97777,50438
97777,51654
97917,51654
97917,52888
98209,52888
98209,54048
97408,54048
97408,55315
97904,55315
97904,56494
97509,56494
97509,57724
97504,57724
97504,58929
97317,58929
97317,60009
96537,60009
96537,61174
96226,61174
96226,62461
96371,62461
96371,63724
96352,63724
96352,64802
95707,64802
95707,66100
95728,66100
95728,67084
94861,67084
94861,68094
94118,68094
94118,69459
94221,69459
94221,70553
93671,70553
93671,71397
92628,71397
92628,72727
92553,72727
92553,73820
92003,73820
92003,74727
91138,74727
91138,75699
90400,75699
90400,77035
90214,77035
90214,77850
89241,77850
89241,78773
88440,78773
88440,79779
87751,79779
87751,80541
86762,80541
86762,81647
86188,81647
86188,82703
85536,82703
85536,83111
84200,83111
84200,84566
83937,84566
83937,84866
82539,84866
82539,85617
81590,85617
81590,86529
80782,86529
80782,87177
79750,87177
79750,88382
79151,88382
79151,88851
77984,88851
77984,89575
77009,89575
77009,90253
76002,90253
76002,90543
74757,90543
74757,91225
73761,91225
7376

In [72]:
codes = []
for line in lines:
    x, y = map(int, line.split(','))
    codes.append((x, y))

print(f"Total red tiles: {len(codes)}")
print(f"First few coordinates: {codes[:5]}")
print(f"Last few coordinates: {codes[-5:]}")

# Find min/max to see the grid dimensions (just for info)
x_coords = [x for x, y in codes]
y_coords = [y for x, y in codes]
print(f"\nX range: {min(x_coords)} to {max(x_coords)}")
print(f"Y range: {min(y_coords)} to {max(y_coords)}")


Total red tiles: 496
First few coordinates: [(97777, 50438), (97777, 51654), (97917, 51654), (97917, 52888), (98209, 52888)]
Last few coordinates: [(98229, 48026), (97538, 48026), (97538, 49226), (97874, 49226), (97874, 50438)]

X range: 1602 to 98229
Y range: 1708 to 98328


In [71]:
max_area = 0
best_pair = None

for i in range(len(codes)):
    for j in range(i + 1, len(codes)):
        x1, y1 = codes[i]
        x2, y2 = codes[j]

        area = (abs(x2 - x1) + 1) * (abs(y2 - y1) + 1)

        if area > max_area:
            max_area = area
            best_pair = (codes[i], codes[j])

print(f'Part 1 max Area: {max_area} from points {best_pair}')


Part 1 max Area: 4741848414 from points ((83937, 84866), (12865, 18149))


In [73]:
# === Verify polygon structure ===

invalid = []
    
for i in range(len(codes)):
    x1, y1 = codes[i]
    x2, y2 = codes[(i + 1) % len(codes)]  # Wrap around
    
    # Check if aligned horizontally or vertically
    if x1 != x2 and y1 != y2:
        invalid.append((i, codes[i], codes[(i + 1) % len(codes)]))

if invalid:
    print(f"Found {len(invalid)} invalid connections:")
    for idx, p1, p2 in invalid[:5]:  # Show first 5
        print(f"  {idx}: {p1} -> {p2} (not aligned)")
else:
    print("All connections are valid! Polygon is properly formed.")

All connections are valid! Polygon is properly formed.


In [74]:
# Create red tiles set
red_tiles_set = set(codes)

# Find bounds
x_coords = [x for x, y in codes]
y_coords = [y for x, y in codes]
min_x, max_x = min(x_coords), max(x_coords)
min_y, max_y = min(y_coords), max(y_coords)

print(f"Red tiles: {len(red_tiles_set)}")
print(f"Grid bounds: x=[{min_x}, {max_x}], y=[{min_y}, {max_y}]")

# Build horizontal and vertical edge segments for O(1) lookup
horizontal_edges = {}  # y -> list of (x_min, x_max)
vertical_edges = {}    # x -> list of (y_min, y_max)

for i in range(len(codes)):
    x1, y1 = codes[i]
    x2, y2 = codes[(i + 1) % len(codes)]
    
    if x1 == x2:  # Vertical edge
        if x1 not in vertical_edges:
            vertical_edges[x1] = []
        vertical_edges[x1].append((min(y1, y2), max(y1, y2)))
    else:  # Horizontal edge
        if y1 not in horizontal_edges:
            horizontal_edges[y1] = []
        horizontal_edges[y1].append((min(x1, x2), max(x1, x2)))

print(f"Built edge lookup: {len(horizontal_edges)} horizontal, {len(vertical_edges)} vertical")

# Cache for point validation
validation_cache = {}

# Fast function to check if point is on polygon edge
def is_on_edge_fast(x, y):
    # Check horizontal edges at this y
    if y in horizontal_edges:
        for x_min, x_max in horizontal_edges[y]:
            if x_min <= x <= x_max:
                return True
    # Check vertical edges at this x
    if x in vertical_edges:
        for y_min, y_max in vertical_edges[x]:
            if y_min <= y <= y_max:
                return True
    return False

# Fast function to check if point is inside polygon (ray casting)
def is_inside_fast(x, y):
    n = len(codes)
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = codes[i]
        xj, yj = codes[j]
        
        if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
            inside = not inside
        
        j = i
    return inside

# Combined check with caching
def is_valid_tile(x, y):
    key = (x, y)
    if key not in validation_cache:
        if key in red_tiles_set:
            validation_cache[key] = True
        elif is_on_edge_fast(x, y):
            validation_cache[key] = True
        else:
            validation_cache[key] = is_inside_fast(x, y)
    return validation_cache[key]



Red tiles: 496
Grid bounds: x=[1602, 98229], y=[1708, 98328]
Built edge lookup: 247 horizontal, 248 vertical


In [75]:
import time

max_area = 0
best_pair = None
checked = 0
valid_rectangles = 0

# Sort pairs by potential area (largest first)
pairs_by_area = []
for i in range(len(codes)):
    for j in range(i + 1, len(codes)):
        x1, y1 = codes[i]
        x2, y2 = codes[j]
        area = (abs(x2 - x1) + 1) * (abs(y2 - y1) + 1)
        pairs_by_area.append((area, i, j))

pairs_by_area.sort(reverse=True)
print(f"Total pairs: {len(pairs_by_area):,}")
print(f"Max possible: {pairs_by_area[0][0]:,}\n")

start_time = time.time()

# Optimized validation: check by rows instead of individual tiles
def is_rectangle_valid_fast(min_x, max_x, min_y, max_y):
    """Check rectangle using scanline optimization"""
    # For narrow rectangles, check every tile
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    
    if width * height <= 10000:  # Small rectangle, just brute force
        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                if not is_valid_tile(x, y):
                    return False
        return True
    
    # For larger rectangles, use sampling strategy
    # Check all 4 corners
    if not (is_valid_tile(min_x, min_y) and is_valid_tile(min_x, max_y) and 
            is_valid_tile(max_x, min_y) and is_valid_tile(max_x, max_y)):
        return False
    
    # Check perimeter with full coverage
    for x in range(min_x, max_x + 1):
        if not is_valid_tile(x, min_y) or not is_valid_tile(x, max_y):
            return False
    
    for y in range(min_y + 1, max_y):
        if not is_valid_tile(min_x, y) or not is_valid_tile(max_x, y):
            return False
    
    # Sample interior strategically - check middle row and column
    mid_x = (min_x + max_x) // 2
    mid_y = (min_y + max_y) // 2
    
    # Check middle row
    for x in range(min_x, max_x + 1, max(1, width // 50)):
        if not is_valid_tile(x, mid_y):
            return False
    
    # Check middle column
    for y in range(min_y, max_y + 1, max(1, height // 50)):
        if not is_valid_tile(mid_x, y):
            return False
    
    # If perimeter and middle samples are valid, likely entire rectangle is valid
    # Do a final diagonal check
    step = max(2, min(width, height) // 20)
    for i in range(0, min(width, height), step):
        if not is_valid_tile(min_x + i, min_y + i):
            return False
    
    return True

for area, i, j in pairs_by_area:
    if area <= max_area:
        print(f"\n⚡ Early exit at {checked:,} - can't beat {max_area:,}")
        break
    
    checked += 1
    x1, y1 = codes[i]
    x2, y2 = codes[j]
    
    min_x, max_x = min(x1, x2), max(x1, x2)
    min_y, max_y = min(y1, y2), max(y1, y2)
    
    if is_rectangle_valid_fast(min_x, max_x, min_y, max_y):
        valid_rectangles += 1
        if area > max_area:
            max_area = area
            best_pair = (codes[i], codes[j])
            elapsed = time.time() - start_time
            print(f"✓ New max: {max_area:,} @ {best_pair} ({elapsed:.1f}s)")
    
    if checked % 1000 == 0:
        elapsed = time.time() - start_time
        print(f"Checked: {checked:,} pairs, Valid: {valid_rectangles:,}, Time: {elapsed:.1f}s")

elapsed = time.time() - start_time

print(f"Time: {elapsed:.1f}s")
print(f"Checked: {checked:,} pairs")
print(f"Valid: {valid_rectangles:,}")
print(f"\nLARGEST AREA: {max_area:,}")
print(f"Between: {best_pair}")


Total pairs: 122,760
Max possible: 4,741,848,414

Checked: 1,000 pairs, Valid: 0, Time: 30.4s
Checked: 2,000 pairs, Valid: 0, Time: 61.4s
Checked: 3,000 pairs, Valid: 0, Time: 76.4s
Checked: 4,000 pairs, Valid: 0, Time: 89.3s
Checked: 5,000 pairs, Valid: 0, Time: 97.9s
Checked: 6,000 pairs, Valid: 0, Time: 104.1s
Checked: 7,000 pairs, Valid: 0, Time: 111.3s
Checked: 8,000 pairs, Valid: 0, Time: 125.0s
Checked: 9,000 pairs, Valid: 0, Time: 139.0s
Checked: 10,000 pairs, Valid: 0, Time: 142.7s
Checked: 11,000 pairs, Valid: 0, Time: 150.1s
Checked: 12,000 pairs, Valid: 0, Time: 152.7s
Checked: 13,000 pairs, Valid: 0, Time: 161.1s
Checked: 14,000 pairs, Valid: 0, Time: 170.5s
Checked: 15,000 pairs, Valid: 0, Time: 178.5s
Checked: 16,000 pairs, Valid: 0, Time: 182.7s
Checked: 17,000 pairs, Valid: 0, Time: 187.8s
Checked: 18,000 pairs, Valid: 0, Time: 193.1s
Checked: 19,000 pairs, Valid: 0, Time: 199.2s
Checked: 20,000 pairs, Valid: 0, Time: 199.9s
Checked: 21,000 pairs, Valid: 0, Time: 204.0